In [14]:
!pip install kaggle

In [15]:

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [16]:
!kaggle competitions download --force -c titanic

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 5.41MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 53.7MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 29.2MB/s]


In [17]:
import pandas as pd
train_df_raw = pd.read_csv('/content/train.csv')
test_df_raw = pd.read_csv('/content/test.csv')

In [18]:
train_df_raw


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [20]:
def do_transformations(t_df, df):
  df = df.drop("Ticket", axis=1)
  df = df.drop("Cabin", axis=1)
  df = df.drop("Name", axis=1)
  df = pd.get_dummies(df, ['Sex','Embarked'])
  df.Age = df.Age.fillna(t_df.Age.mean())
  df.Fare = df.Fare.fillna(t_df.Fare.mean())
  
  return df

In [21]:
train_df = do_transformations(train_df_raw, train_df_raw)
test_df = do_transformations(train_df_raw, test_df_raw)

train_df = train_df.drop("PassengerId", axis=1)

In [22]:
train_df

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.000000,1,0,7.2500,0,1,0,0,1
1,1,1,38.000000,1,0,71.2833,1,0,1,0,0
2,1,3,26.000000,0,0,7.9250,1,0,0,0,1
3,1,1,35.000000,1,0,53.1000,1,0,0,0,1
4,0,3,35.000000,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.000000,0,0,13.0000,0,1,0,0,1
887,1,1,19.000000,0,0,30.0000,1,0,0,0,1
888,0,3,29.699118,1,2,23.4500,1,0,0,0,1
889,1,1,26.000000,0,0,30.0000,0,1,1,0,0


In [23]:
from sklearn.tree import DecisionTreeClassifier

X = train_df[train_df.columns[1:]]
y = train_df["Survived"]

model = DecisionTreeClassifier()
model.fit(X,y)

best_model = model


In [24]:
def generate_submission_file(best_model, test_df):
  test_df2 = test_df.drop('PassengerId', axis=1)
  Y = best_model.predict(test_df2)
  
  
  submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": Y
  })
  submission.to_csv('/content/submission.csv', index=False)

In [25]:
generate_submission_file(best_model, test_df)

In [26]:
!kaggle competitions submit -c titanic -f /content/submission.csv -m "A test submission"

  0% 0.00/2.77k [00:00<?, ?B/s]User cancelled operation
